In [54]:
!pip install pytesseract pillow pdf2image openai reportlab requests

In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample-report.pdf to sample-report (2).pdf


In [3]:
import os
os.listdir()

['.config',
 'sample-report (1).pdf',
 'sample-report (2).pdf',
 'Final_Report.pdf',
 'sample-report.pdf',
 'sample_data']

In [4]:
from pdf2image import convert_from_path

images = convert_from_path("sample-report.pdf")
print(len(images))

3


In [5]:
!apt-get update
!apt-get install -y poppler-utils

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,632 B in 1s (3,685 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02

In [6]:
import pytesseract

raw_text = ""

for img in images:
    raw_text += pytesseract.image_to_string(img)

print(raw_text[:1000])

CAR LEASE AGREEMENT

This Car Lease Agreement (“Agreement”) is entered into on 15 March 2025 (Effective Date)
by and between:

Lessor: ABC Auto Leasing Pvt. Ltd.
Address: 45 Industrial Road, Andheri East, Mumbai, Maharashtra — 400069, India

Lessee: Ansh Verma
Address: Flat 12B, Shanti Apartments, Sector 21, Noida, Uttar Pradesh — 201301, India

1. VEHICLE DETAILS
Make: Toyota
Model: Corolla Altis
Year: 2023
Vehicle Identification Number (VIN): JIDBR32E720123456
Color: White

2. LEASE TERM
Lease Start Date: 20 March 2025
Lease End Date: 19 March 2028
Lease Term Duration: 36 months

3. FINANCIAL TERMS

3.1 Capitalized Cost: %1,850,000

3.2 Interest Rate / APR: 8.5% per annum
3.3 Monthly Lease Payment: %42,500

3.4 Payment Due Date: 5th of each month
3.5 Down Payment: %150,000

3.6 Security Deposit: 350,000

4. MILEAGE ALLOWANCE
Annual Mileage Allowance: 15,000 km per year
Excess Mileage Charge: 312 per km

5. RESIDUAL VALUE
Residual Value at Lease End: 900,000
6. MAINTENANCE AND REPAIR

In [7]:
clean_text = raw_text.replace("\n\n", "\n").strip()

In [8]:
sla_schema = """
{
  "lessee_name": "",
  "lessor_name": "",
  "vehicle_make": "",
  "vehicle_model": "",
  "vehicle_year": "",
  "vin": "",
  "lease_term_months": "",
  "interest_rate_apr": "",
  "monthly_payment": "",
  "down_payment": "",
  "annual_mileage_limit": "",
  "early_termination_fee": "",
  "late_payment_fee": "",
  "buyout_price": ""
}
"""

In [39]:
prompt = f"""
You are a strict JSON generator.

Return ONLY valid JSON.
DO NOT add explanations.
DO NOT add markdown.
DO NOT add text before or after JSON.

Use this schema exactly:
{sla_schema}

If a value is missing, write "Not Found".

Contract Text:
{clean_text}
"""

In [40]:
from openai import OpenAI

client = OpenAI(api_key="OPENAI_API_KEY")

In [42]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

sla_output = response.choices[0].message.content
print(sla_output)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: PASTE_YO***********HERE. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [43]:
import json
sla_data = json.loads(sla_output)
sla_data

{'lessee_name': 'Ansh Verma',
 'lessor_name': 'ABC Auto Leasing Pvt. Ltd.',
 'vehicle_make': 'Toyota',
 'vehicle_model': 'Corolla Altis',
 'vehicle_year': '2023',
 'vin': 'JIDBR32E720123456',
 'lease_term_months': '36',
 'interest_rate_apr': '8.5%',
 'monthly_payment': '42500',
 'down_payment': '150000',
 'annual_mileage_limit': '15000 km',
 'early_termination_fee': '75000',
 'late_payment_fee': '31500',
 'buyout_price': '900000'}

In [44]:
print("----- RAW AI OUTPUT START -----")
print(sla_output)
print("----- RAW AI OUTPUT END -----")

----- RAW AI OUTPUT START -----
{
  "lessee_name": "Ansh Verma",
  "lessor_name": "ABC Auto Leasing Pvt. Ltd.",
  "vehicle_make": "Toyota",
  "vehicle_model": "Corolla Altis",
  "vehicle_year": "2023",
  "vin": "JIDBR32E720123456",
  "lease_term_months": "36",
  "interest_rate_apr": "8.5%",
  "monthly_payment": "42500",
  "down_payment": "150000",
  "annual_mileage_limit": "15000 km",
  "early_termination_fee": "75000",
  "late_payment_fee": "31500",
  "buyout_price": "900000"
}
----- RAW AI OUTPUT END -----


In [45]:
def extract_json(text):
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1:
        raise ValueError("No JSON object found in AI output")
    return text[start:end+1]

clean_json = extract_json(sla_output)
print(clean_json)

{
  "lessee_name": "Ansh Verma",
  "lessor_name": "ABC Auto Leasing Pvt. Ltd.",
  "vehicle_make": "Toyota",
  "vehicle_model": "Corolla Altis",
  "vehicle_year": "2023",
  "vin": "JIDBR32E720123456",
  "lease_term_months": "36",
  "interest_rate_apr": "8.5%",
  "monthly_payment": "42500",
  "down_payment": "150000",
  "annual_mileage_limit": "15000 km",
  "early_termination_fee": "75000",
  "late_payment_fee": "31500",
  "buyout_price": "900000"
}


In [46]:
import json

sla_data = json.loads(clean_json)
sla_data

{'lessee_name': 'Ansh Verma',
 'lessor_name': 'ABC Auto Leasing Pvt. Ltd.',
 'vehicle_make': 'Toyota',
 'vehicle_model': 'Corolla Altis',
 'vehicle_year': '2023',
 'vin': 'JIDBR32E720123456',
 'lease_term_months': '36',
 'interest_rate_apr': '8.5%',
 'monthly_payment': '42500',
 'down_payment': '150000',
 'annual_mileage_limit': '15000 km',
 'early_termination_fee': '75000',
 'late_payment_fee': '31500',
 'buyout_price': '900000'}

In [47]:
import requests

vin = sla_data["vin"]

url = f"https://vpic.nhtsa.dot.gov/api/vehicles/DecodeVin/{vin}?format=json"
vin_data = requests.get(url).json()

vin_data["Results"][:5]

[{'Value': 'J!DBR32E720123456',
  'ValueId': '',
  'Variable': 'Suggested VIN',
  'VariableId': 142},
 {'Value': '1,7,400',
  'ValueId': '1,7,400',
  'Variable': 'Error Code',
  'VariableId': 143},
 {'Value': '',
  'ValueId': '',
  'Variable': 'Possible Values',
  'VariableId': 144},
 {'Value': 'Invalid character(s): 2:I.',
  'ValueId': '',
  'Variable': 'Additional Error Text',
  'VariableId': 156},
 {'Value': '1 - Check Digit (9th position) does not calculate properly; 7 - Manufacturer is not registered with NHTSA for sale or importation in the U.S. for use on U.S roads; Please contact the manufacturer directly for more information; 400 - Invalid Characters Present',
  'ValueId': '',
  'Variable': 'Error Text',
  'VariableId': 191}]

In [48]:
base_price_table = {
    "Toyota Corolla Altis": 1800000,
    "Honda City": 1700000,
    "Hyundai Verna": 1600000,
    "Maruti Swift": 800000
}

In [49]:
def estimate_market_price(make, model, year):
    key = f"{make} {model}"

    # base price
    base_price = base_price_table.get(key, 1500000)  # default

    current_year = 2025
    age = current_year - year

    # depreciation: 5% per year
    depreciation_rate = 0.05
    final_price = base_price * (1 - depreciation_rate * age)

    return int(final_price)

In [52]:
estimated_market_price = 1800000
contract_price = 1850000

price_diff = contract_price - estimated_market_price
price_diff

50000

In [53]:
score = 100

if price_diff > 0:
    score -= 10

if "75,000" in sla_data["early_termination_fee"]:
    score -= 15

score

90

In [50]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas

c = canvas.Canvas("Final_Report.pdf", pagesize=A4)
text = c.beginText(40, 800)

text.textLine("AI CAR LEASE CONTRACT REVIEW REPORT")
text.textLine("")
text.textLine(f"Lessee: {sla_data['lessee_name']}")
text.textLine(f"Vehicle: {sla_data['vehicle_make']} {sla_data['vehicle_model']}")
text.textLine(f"Interest Rate: {sla_data['interest_rate_apr']}")
text.textLine(f"Monthly Payment: {sla_data['monthly_payment']}")
text.textLine(f"Fairness Score: {score}/100")

c.drawText(text)
c.save()

In [51]:
from google.colab import files
files.download("Final_Report.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>